# Importação de pacotes

In [2]:
from transformers import BertTokenizer, BertModel
import torch

In [3]:
from pprint import pprint

# Tokenizacao

In [4]:
# Carrega o tokenizer e o modelo pré-treinado do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Modelo base em inglês (sem distinção entre maiúsculas e minúsculas)
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [21]:
sentenca = "The world has been changing very quickly."

In [22]:
input_ids = tokenizer.encode(sentenca, add_special_tokens=True)
input_ids

[101, 1996, 2088, 2038, 2042, 5278, 2200, 2855, 1012, 102]

In [24]:
# Converte os IDs dos tokens para tensores
input_ids = torch.tensor([input_ids])
input_ids

tensor([[ 101, 1996, 2088, 2038, 2042, 5278, 2200, 2855, 1012,  102]])

# Obtencao Embedding Bert

In [25]:

with torch.no_grad():
    outputs = model(input_ids)

A frase (sequência) de entrada contêm 7 palavras e ao ser tokenizada e adicionado tokens especiais que orientam o modelo como (CLS,SEP,PAD ou até mesmo UNK), geram um tensor com 10 palavras devido a adição do tokens especiais que também serão contabilizados.

In [9]:
pprint(outputs)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4455,  0.2237, -0.2114,  ..., -0.5643,  0.3196,  0.6761],
         [ 0.3459, -0.2484, -0.5160,  ..., -0.3459,  0.9520,  0.3300],
         [ 0.4154, -0.2148,  0.1933,  ..., -0.5308,  0.2658, -0.0108],
         ...,
         [-0.0197, -0.5172, -0.0060,  ..., -0.6253,  0.1119, -0.0759],
         [ 1.0998,  0.1138, -0.3302,  ..., -0.0840, -0.5815, -0.4058],
         [ 0.9522,  0.1612,  0.1625,  ...,  0.1789, -0.7191, -0.3716]]]),
                                             pooler_output=tensor([[-9.0126e-01, -4.3130e-01, -9.1306e-01,  7.9034e-01,  6.5514e-01,
         -2.3006e-01,  7.9152e-01,  3.0719e-01, -8.8875e-01, -9.9993e-01,
         -6.9544e-01,  9.8311e-01,  9.7954e-01,  4.5719e-01,  9.1757e-01,
         -6.4022e-01, -3.8710e-01, -5.7138e-01,  2.9374e-01, -1.9666e-01,
          6.1739e-01,  9.9998e-01, -7.2110e-02,  4.5631e-01,  5.1987e-01,
          9.9635e-01, -7.8481e-01,  9.3011e-01,  9.3923e-01,  6.3

## last_hidden_state

In [10]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

* last_hidden_state (Último Estado Oculto)

  Formato: É um tensor 3D de tamanho (batch_size, sequence_length, hidden_size).
  
  * batch_size: Número de sequências processadas em lote. no caso abaixo contem apenas 1 batch
  * sequence_length: Comprimento da sequência de entrada (número de tokens), a entrada original possui 7 palavras e ao ser tokenizada passsou a ter 10 tokens que é composo pelo token da palvras + os tokens especiais.
  * hidden_size: Dimensionalidade do embedding (768 para o BERT base).



* Uso: É usado para tarefas que requerem representações de cada palavra no contexto da frase, como:
  * Marcação de sequência (Sequence Tagging): Ex: Part-of-Speech tagging, Named Entity Recognition.
  * Extração de características (Feature Extraction): Usar os embeddings como entrada para outros modelos.
  * Similaridade semântica de palavras: Comparar os embeddings de palavras diferentes.
  * Resposta a perguntas (Question Answering): Identificar a resposta em um contexto.


O tensor retornado outputs['last_hidden_state'].shape é 3d e  o primerio elemento 1 representa o tamanho do batch, o segundo elemento quantidade de palavras na sentença 10 e o terceiro 768 a quantidade de elemento em cada vetor de palavras que correponde ao tipo de modelo utilizado large(1024) ou base(768)

In [11]:
outputs['last_hidden_state'].shape

torch.Size([1, 10, 768])

## embedding como um tensor 2d pooler_output

In [12]:
# Os embeddings estão em outputs[0]
embeddings = outputs[0]
embeddings

tensor([[[ 0.4455,  0.2237, -0.2114,  ..., -0.5643,  0.3196,  0.6761],
         [ 0.3459, -0.2484, -0.5160,  ..., -0.3459,  0.9520,  0.3300],
         [ 0.4154, -0.2148,  0.1933,  ..., -0.5308,  0.2658, -0.0108],
         ...,
         [-0.0197, -0.5172, -0.0060,  ..., -0.6253,  0.1119, -0.0759],
         [ 1.0998,  0.1138, -0.3302,  ..., -0.0840, -0.5815, -0.4058],
         [ 0.9522,  0.1612,  0.1625,  ...,  0.1789, -0.7191, -0.3716]]])

* Formato: É um tensor 2D de tamanho (batch_size, hidden_size).
 * batch_size: Número de sequências processadas em lote.
 * hidden_size: Dimensionalidade do embedding (ex: 768 para o BERT base).
 * Conteúdo: É derivado do embedding do token [CLS] presente no last_hidden_state. O embedding do [CLS] passa por uma camada linear (com função de ativação tanh) para gerar o pooler_output.
 * Uso: É usado principalmente para tarefas de classificação de sequências ou para obter uma representação vetorial única da sequência inteira, como:
 * Classificação de texto: Ex: análise de sentimentos, classificação de tópicos.
Inferência de linguagem natural (NLI): Determinar a relação entre duas frases (entailment, contradiction, neutral).


In [14]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [16]:
embeddings_pooler = outputs[1]
embeddings_pooler

tensor([[-9.0126e-01, -4.3130e-01, -9.1306e-01,  7.9034e-01,  6.5514e-01,
         -2.3006e-01,  7.9152e-01,  3.0719e-01, -8.8875e-01, -9.9993e-01,
         -6.9544e-01,  9.8311e-01,  9.7954e-01,  4.5719e-01,  9.1757e-01,
         -6.4022e-01, -3.8710e-01, -5.7138e-01,  2.9374e-01, -1.9666e-01,
          6.1739e-01,  9.9998e-01, -7.2110e-02,  4.5631e-01,  5.1987e-01,
          9.9635e-01, -7.8481e-01,  9.3011e-01,  9.3923e-01,  6.3363e-01,
         -6.5515e-01,  3.4226e-01, -9.9118e-01, -8.0106e-02, -8.9089e-01,
         -9.8625e-01,  5.1739e-01, -5.2843e-01,  1.6789e-01,  1.1128e-01,
         -8.9169e-01,  3.3362e-01,  9.9993e-01, -1.9495e-01,  3.7232e-01,
         -2.8989e-01, -1.0000e+00,  3.4560e-01, -8.1868e-01,  9.4549e-01,
          8.8624e-01,  9.3882e-01,  1.8720e-01,  5.1905e-01,  5.4005e-01,
         -3.0180e-01, -1.5271e-01,  1.2568e-01, -2.0411e-01, -5.6608e-01,
         -6.7204e-01,  3.7352e-01, -8.6722e-01, -8.3979e-01,  9.4617e-01,
          8.2125e-01, -1.0662e-01, -2.

In [17]:
# Imprime o tamanho dos embeddings
print("Tamanho dos embeddings:", embeddings_pooler.shape)

Tamanho dos embeddings: torch.Size([1, 768])


# Análises

In [18]:
# Imprime o embedding do token [CLS] (primeiro token)
cls_embedding = embeddings[0][0]
print("\nEmbedding do [CLS]:", cls_embedding[:10]) # Imprime os 10 primeiros valores do vetor


Embedding do [CLS]: tensor([ 0.4455,  0.2237, -0.2114, -0.2501, -0.3274, -0.5042,  0.2624,  1.1734,
         0.1072, -0.6908])


In [19]:
# Imprime o embedding do token "cat" (terceiro token)
has_embedding = embeddings[0][2]
print("\nEmbedding de 'has':", has_embedding[:10]) # Imprime os 10 primeiros valores do vetor


Embedding de 'has': tensor([ 0.4154, -0.2148,  0.1933, -0.6385,  0.6478,  0.0321,  0.0640,  1.6508,
        -0.6634, -0.4843])


In [20]:

# Converte os IDs dos tokens para palavras
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print("\nTokens:", tokens)


Tokens: ['[CLS]', 'the', 'world', 'has', 'been', 'changing', 'very', 'quickly', '.', '[SEP]']
